In [2]:
pages_data = [
  {
    "page": 1,
    "story_text": "Tara was having a picnic with her parents when a colorful parrot invited her to explore the jungle.",
    "illustration_description": "Cartoon-style illustration. Tara, a 6-year-old girl with shoulder-length brown hair and big brown eyes, wears a yellow dress with white flower prints and red sandals. She sits on a red picnic blanket under a large tree. Her mother (blue sari, bun) and father (white shirt, jeans) are in the background. Rio, a green parrot with red wing tips, blue tail feathers, and a golden beak, hovers nearby. Style: soft pastel, rounded features, gentle outlines."
  },
  {
    "page": 2,
    "story_text": "Tara followed the parrot into the jungle. 'What a beautiful place!' she exclaimed.",
    "illustration_description": "Tara walks along a jungle path with Rio flying ahead. The jungle is bright and cartoonish, full of oversized leaves, vines, colorful flowers, and playful animals. Sunlight beams through the trees. Tara looks amazed. Style: soft pastel cartoon with playful child-friendly shapes."
  },
  {
    "page": 3,
    "story_text": "The parrot introduced Tara to a friendly tiger who was taking a nap under a tree.",
    "illustration_description": "Tara stands in a sunlit clearing. A friendly tiger, Raja, with orange fur and soft black stripes, stretches and yawns under a tree. Tara smiles, Rio perches above on a branch. Jungle foliage in the background. Style: gentle cartoon, warm colors, rounded animal design."
  },
  {
    "page": 4,
    "story_text": "'Hello,' said the tiger with a sleepy smile. 'Would you like to see the magical waterfall?'",
    "illustration_description": "The tiger sits and speaks to Tara, who looks excited. Rio flutters nearby. The scene is cozy with green leaves, jungle flowers, and a sparkling brook in the background. Characters drawn in a soft, rounded cartoon style with warm facial expressions."
  },
  {
    "page": 5,
    "story_text": "They walked through the jungle together. The parrot flew from tree to tree, showing them the way.",
    "illustration_description": "Tara walks beside Raja the tiger while Rio flies from tree to tree, guiding the way. Jungle trees arch above them with birds and butterflies fluttering. Tara looks joyful. Cartoon-style art with childlike whimsy and colorful, dreamy jungle elements."
  },
  {
    "page": 6,
    "story_text": "Soon they arrived at a beautiful waterfall with rainbow colors dancing in the mist.",
    "illustration_description": "A magical waterfall flows down sparkling rocks into a rainbow mist. Tara gazes in wonder with arms outstretched. Raja stands nearby and Rio circles in the air. The water glows, and rainbow light sparkles. Cartoon fantasy style with soft gradients and gentle expressions."
  },
  {
    "page": 7,
    "story_text": "'This is magical!' said Tara. She thanked her new friends for showing her such a wonderful place.",
    "illustration_description": "Tara hugs Rio while Raja smiles proudly. They stand near the waterfall with rainbows glowing in the mist behind them. The jungle glistens softly. Emotions are happy and grateful. Style: heartwarming cartoon with gentle, rounded design."
  },
  {
    "page": 8,
    "story_text": "As the sun began to set, the parrot and tiger guided Tara back to the picnic area where her parents were waiting.",
    "illustration_description": "Tara walks between Raja and Rio, heading back through the jungle bathed in sunset light. Orange and pink sky behind the trees. Her parents stand up in the distance, smiling. Cartoon sunset scene with soft colors and warm family reunion tone."
  },
  {
    "page": 9,
    "story_text": "'Thank you for the adventure,' Tara said as she hugged her new friends goodbye. 'Will I see you again?'",
    "illustration_description": "Tara hugs both Rio and Raja. She looks hopeful and happy. Her parents pack up the picnic in the background. The sky is dusky purple with twinkling stars. Style: soft emotional cartoon illustration with gentle glow effects."
  },
  {
    "page": 10,
    "story_text": "That night, Tara dreamed of her jungle adventure. On her bedside table sat a colorful feather—a gift from her parrot friend.",
    "illustration_description": "Tara sleeps peacefully in her cozy bedroom. A soft nightlight glows. On the bedside table is Rio’s colorful feather. Dream bubbles above her head show jungle scenes. Style: warm pastel cartoon with dreamy, magical bedtime feel."
  }
]


In [4]:
import json
import requests
import time
import os
from datetime import datetime

class LeonardoStoryGenerator:
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": f"Bearer {self.api_key}"
        }
        # Create output directory for saving images
        self.output_dir = "story_illustrations_2"
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Character reference ID for consistency
        self.character_reference_id = None
        
    def upload_image(self, image_path):
        """Upload an image to Leonardo.ai and return the image ID"""
        # Get a presigned URL for uploading
        url = "https://cloud.leonardo.ai/api/rest/v1/init-image"
        extension = image_path.split('.')[-1]
        payload = {"extension": extension}
        
        response = requests.post(url, json=payload, headers=self.headers)
        if response.status_code != 200:
            print(f"Failed to get presigned URL: {response.text}")
            return None
            
        upload_data = response.json()['uploadInitImage']
        fields = json.loads(upload_data['fields'])
        upload_url = upload_data['url']
        uploaded_image_id = upload_data['id']
        
        # Upload the image to the presigned URL
        with open(image_path, 'rb') as image_file:
            files = {'file': image_file}
            response = requests.post(upload_url, data=fields, files=files)
            
        if response.status_code != 204:
            print(f"Failed to upload image: {response.status_code}")
            return None
            
        print(f"Successfully uploaded image with ID: {uploaded_image_id}")
        return uploaded_image_id
    
    def generate_image(self, prompt, width=1024, height=768, style_image_id=None, character_reference_id=None):
        """Generate an image with Leonardo.ai"""
        url = "https://cloud.leonardo.ai/api/rest/v1/generations"
        
        # Base payload
        payload = {
            "height": height,
            "width": width,
            "modelId": "aa77f04e-3eec-4034-9c07-d0f619684628",  # Leonardo Kino XL
            "prompt": prompt,
            "presetStyle": "ANIME",
            "photoReal": True,
            "photoRealVersion": "v2",
            "alchemy": True,
        }
        
        # Add controlnets if we have reference images
        if character_reference_id or style_image_id:
            payload["controlnets"] = []
            
            if character_reference_id:
                payload["controlnets"].append({
                    "initImageId": character_reference_id,
                    "initImageType": "UPLOADED",
                    "preprocessorId": 133,  # Character Reference Id
                    "strengthType": "High",  # Increased from Mid to High for better consistency
                })
                
            if style_image_id:
                payload["controlnets"].append({
                    "initImageId": style_image_id,
                    "initImageType": "GENERATED",
                    "preprocessorId": 67,  # Style Reference Id
                    "strengthType": "High",
                })
        
        response = requests.post(url, json=payload, headers=self.headers)
        if response.status_code != 200:
            print(f"Failed to create generation: {response.text}")
            return None
            
        generation_id = response.json()['sdGenerationJob']['generationId']
        print(f"Initiated generation with ID: {generation_id}")
        
        # Wait for the generation to complete
        url = f"https://cloud.leonardo.ai/api/rest/v1/generations/{generation_id}"
        
        # Poll until the generation is complete
        max_attempts = 30
        for attempt in range(max_attempts):
            print(f"Waiting for generation to complete... Attempt {attempt + 1}/{max_attempts}")
            time.sleep(10)  # Wait 10 seconds between checks
            
            response = requests.get(url, headers=self.headers)
            if response.status_code != 200:
                print(f"Failed to get generation status: {response.text}")
                continue
                
            generation_data = response.json()['generations_by_pk']
            status = generation_data['status']
            
            if status == "COMPLETE":
                image_urls = [img['url'] for img in generation_data['generated_images']]
                image_ids = [img['id'] for img in generation_data['generated_images']]
                
                if image_urls and image_ids:
                    print(f"Generation complete! Images available at: {image_urls[0]}")
                    return {
                        "url": image_urls[0],
                        "id": image_ids[0]
                    }
                    
            elif status == "FAILED":
                print("Generation failed!")
                return None
        
        print("Generation timed out!")
        return None
    
    def download_image(self, image_url, output_path):
        """Download an image from a URL"""
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded image to {output_path}")
            return True
        else:
            print(f"Failed to download image: {response.status_code}")
            return False
    
    def create_consistent_prompt(self, page_data, character_description):
        """Create a consistent prompt that reinforces character appearance on every page"""
        # Start with the base style instruction for consistency
        consistent_style = """Soft cartoon anime-style jungle illustration with pastel tones and gentle, rounded shapes.
                        Characters should have big, expressive anime-style eyes, warm smiles, and child-friendly proportions. Use smooth outlines and soft color gradients with minimal detailing to keep the look gentle and cozy."""
        
        # Add the specific illustration description for this page
        illustration = page_data['illustration_description']
        
        # Combine everything into a final prompt
        final_prompt = f"""{consistent_style}
                        STORY: {page_data['story_text']}
                        
                        CHARACTER REFERENCE: Tara – A 6-year-old girl with shoulder-length wavy brown hair, big sparkling brown anime-style eyes, and light skin. She wears a yellow dress with small white flower prints, red sandals, and carries a light brown sling bag with jungle animal prints. Her expression is joyful and curious.
                        
                        ILLUSTRATION DESCRIPTION: {illustration}

                        The scene should be brightly colored, cheerful, with clean lines. Maintain character consistency across all illustrations."""

        return final_prompt
        
    def generate_story_illustrations(self, pages_data, use_style_reference=True):
        """Generate illustrations for every page of the story"""
        print("Starting story illustration generation...")
        
        # First, generate an art style reference image
        style_reference = None
        if use_style_reference:
            print("Generating art style reference...")
            style_result = self.generate_image(
                prompt="""Soft cartoon anime-style jungle illustration with pastel tones and gentle, rounded shapes.
                        Characters should have big, expressive anime-style eyes, warm smiles, and child-friendly proportions. Use smooth outlines and soft color gradients with minimal detailing to keep the look gentle and cozy.
                        The jungle background should feel magical and whimsical — glowing green leaves, oversized tropical plants, twisting vines, soft sunbeams filtering through trees, and occasional colorful jungle animals.
                        Include dreamy visual touches like sparkles in the mist, floating petals, or softly glowing fireflies.
                        Avoid sharp lines or high contrast. The mood should be playful, peaceful, and imaginative, perfect for a children’s storybook set in a magical jungle.
                        Style inspiration: a blend of pastel-toned Japanese children’s anime and storybook-style illustrations — consistent, cute, and age-appropriate for 4–8 year olds.""",
                height=576,
                width=512
            )
            if style_result:
                style_reference = style_result["id"]
                print(f"Created style reference with ID: {style_reference}")
                
                # Download the style reference image
                timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
                style_ref_path = os.path.join(self.output_dir, f"style_reference_{timestamp}.png")
                self.download_image(style_result["url"], style_ref_path)
        
        # Generate the first page illustration
        print("\n--- Generating Page 1 Illustration ---")
        first_page = pages_data[0]
        
        
        # Create a consistent prompt for the first page
        character_prompt = """
        Create a cartoon anime-style character reference set in a whimsical jungle.
        Include three main characters in the same frame:

        Tara – A 6-year-old girl with shoulder-length wavy brown hair, big sparkling brown anime-style eyes, and light skin. She wears a yellow dress with small white flower prints, red sandals, and carries a light brown sling bag with jungle animal prints. Her expression is joyful and curious.

        The background should be a light-filled, magical jungle scene with glowing leaves, soft beams of sunlight, and oversized cartoonish plants.
        The overall art style should blend cute anime features (like big eyes, soft color gradients, expressive faces) with soft cartoon outlines. Use pastel tones, soft shadows, and a warm atmosphere suitable for a children’s storybook.
        """
        
        character_result = self.generate_image(
            prompt=character_prompt,
            style_image_id=style_reference,
        )
        
        if not character_result:
            print("Failed to generate first page illustration. Exiting.")
            return
            
        # Download the first page image
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        first_page_path = os.path.join(self.output_dir, f"character_{timestamp}.png")
        self.download_image(character_result["url"], first_page_path)
        
        # Upload the first page image to use as character reference
        character_reference_id = self.upload_image(first_page_path)
        if not character_reference_id:
            print("Failed to upload character reference. Exiting.")
            return
            
        print(f"Successfully created character reference with ID: {character_reference_id}")
        
        # Store all generated images for reference
        generated_images = [first_page_path]
        
        # Generate illustrations for the remaining pages using the character reference
        for i, page in enumerate(pages_data, start=1):
            print(f"\n--- Generating Page {i} Illustration ---")
            
            # Create a consistent prompt for this page
            page_prompt = self.create_consistent_prompt(page, character_prompt)
            
            # Generate the image for this page
            page_result = self.generate_image(
                prompt=page_prompt,
                character_reference_id=character_reference_id,
                style_image_id=style_reference
            )
            
            if page_result:
                # Download the page image
                page_path = os.path.join(self.output_dir, f"page_{i:02d}_{timestamp}.png")
                self.download_image(page_result["url"], page_path)
                generated_images.append(page_path)
                
                # Every 3 pages, update the character reference to maintain consistency
                # while adapting to any evolving scene elements
                # if i % 3 == 0:
                #     print(f"Updating character reference after page {i}")
                #     character_reference_id = self.upload_image(page_path)
                #     if character_reference_id:
                #         print(f"Updated character reference ID: {character_reference_id}")
                #     else:
                #         print("Failed to update character reference, using previous reference")
            else:
                print(f"Failed to generate illustration for page {i}")
        
        print("\nStory illustration generation complete!")
        print(f"Generated {len(generated_images)} illustrations out of {len(pages_data)} pages")
        return generated_images
        
# Main execution
if __name__ == "__main__":
    # Your API key
    api_key = "e2c8696a-fec6-438b-8071-e705f86eeba9"  # Replace with your actual API key

    # Create a generator instance and generate illustrations
    generator = LeonardoStoryGenerator(api_key)
    generator.generate_story_illustrations(pages_data)

Starting story illustration generation...
Generating art style reference...
Initiated generation with ID: 6b119944-9940-4c3d-9ff3-d7fe5f350767
Waiting for generation to complete... Attempt 1/30
Waiting for generation to complete... Attempt 2/30
Generation complete! Images available at: https://cdn.leonardo.ai/users/a62199bf-f394-4cf4-8cb0-eda30eea151a/generations/6b119944-9940-4c3d-9ff3-d7fe5f350767/Leonardo_Kino_XL_Soft_cartoon_animestyle_jungle_illustration_w_0.jpg
Created style reference with ID: 0200c9e5-217e-4755-be66-59565b8e2575
Downloaded image to story_illustrations_2/style_reference_20250427032832.png

--- Generating Page 1 Illustration ---
Initiated generation with ID: eeae34fb-2216-4d92-a6a6-5b3ce077060b
Waiting for generation to complete... Attempt 1/30
Waiting for generation to complete... Attempt 2/30
Waiting for generation to complete... Attempt 3/30
Generation complete! Images available at: https://cdn.leonardo.ai/users/a62199bf-f394-4cf4-8cb0-eda30eea151a/generations/